In [ ]:
# Correlation Analysis
# Run the code in a cell below by selecting it and clicking on the play button at the top
# For the code to run smoothly, run each cell in order

In [ ]:
%matplotlib inline
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [ ]:
# Enter the ticker symbol of the stock you're interested in
ticker = "ITW"

In [ ]:
price_data_1 = pd.read_csv("FundPerformance.csv")

In [ ]:
price_data_2 = yf.download(ticker, start = "2010-01-01")

In [ ]:
price_data_1 = price_data_1.set_index("Date")

In [ ]:
price_data_1["SSIF Portfolio"] = price_data_1["SSIF Portfolio"]/100 + 1

In [ ]:
price_data_1.index = pd.to_datetime(price_data_1.index)

In [ ]:
combined_data = pd.merge(price_data_1["SSIF Portfolio"], price_data_2["Adj Close"], left_index = True, right_index = True)

In [ ]:
# Take log returns of daily price change
adj_data_1 = np.log(combined_data.iloc[1:]["SSIF Portfolio"].values/combined_data.iloc[:-1]["SSIF Portfolio"].values)
adj_data_2 = np.log(combined_data.iloc[1:]["Adj Close"].values/combined_data.iloc[:-1]["Adj Close"].values)

In [ ]:
combined_data.loc[combined_data.index[1]:, "SSIF Portfolio"] = adj_data_1
combined_data.loc[combined_data.index[1]:, "Adj Close"] = adj_data_2
combined_data = combined_data.dropna().loc[combined_data.index[1]:,:]

In [ ]:
combined_data.corr()

In [ ]:
# Calculate trailing 6-month correlation between the SSIF and the stock
times = combined_data.index
year = datetime.timedelta(days = 182)
corrs = []
for i in combined_data[times[0] + year:].index:
    corrs.append([i,combined_data[i - year:i].corr().iloc[0][1]])


In [ ]:
corrs = np.array(corrs)
corrs

In [ ]:
# Plot correlation coefficient over last 2 years
fig = plt.figure(figsize = (12,6))
ax = plt.axes()

ax.set_ylabel("Correlation Coefficient")
ax.yaxis.label.set_color("black")
ax.yaxis.label.set_size(18)

ax.tick_params(axis = 'x', colors = "black")
ax.tick_params(axis = 'y', colors = "black")

ax.plot(corrs[:,0], corrs[:,1]);

In [ ]:
# Plot price chart of the SSIF and the stock
combined_data = pd.merge(price_data_1["SSIF Portfolio"], price_data_2["Adj Close"], left_index = True, right_index = True)

fig = plt.figure(figsize = (14,8))
ax = plt.axes()

ax.tick_params(axis = 'x', colors = "white")
ax.tick_params(axis = 'y', colors = "white")

ax.plot(combined_data["SSIF Portfolio"]/(combined_data.iloc[0]["SSIF Portfolio"]/combined_data.iloc[0]["Adj Close"]), label = "SSIF Portfolio")
ax.plot(combined_data["Adj Close"], label = ticker)

leg = ax.legend();